In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import csv

import json
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim 
from geopy.exc import GeocoderTimedOut

print('Libraries imported.')

Libraries imported.


###### Extract the HTML code of the source website

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_Sydney_Trains_railway_stations').text
soup = BeautifulSoup(source, 'lxml')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Sydney Trains railway stations - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b4bdf621-4674-4ef6-b6bb-a6c607ee077b","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Sydney_Trains_railway_stations","wgTitle":"List of Sydney Trains railway stations","wgCurRevisionId":962828593,"wgRevisionId":962828593,"wgArticleId":537243,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles with short description","Use dmy

###### After detailed observation of the HTML code, you can see that all the suburb names are in bold tags. So we can extract all the paragraph tags and then filter out the extra, unnecessary data.

In [3]:
stations=[]
for i in soup.select('b'):
    stations.append(i.getText()) 
stations

['Allawah',
 'Arncliffe',
 'Artarmon',
 'Ashfield',
 'Asquith',
 'Auburn',
 'Banksia',
 'Bankstown',
 'Bardwell Park',
 'Beecroft',
 'Belmore',
 'Berala',
 'Berowra',
 'Beverly Hills',
 'Bexley North',
 'Birrong',
 'Blacktown',
 'Major Interchange',
 'Bondi Junction',
 'Burwood',
 'Cabramatta',
 'Campbelltown',
 'Major Interchange',
 'Campsie',
 'Canley Vale',
 'Canterbury',
 'Caringbah',
 'Carlton',
 'Carramar',
 'Casula',
 'Central',
 'Major Interchange',
 'Chatswood',
 'Major Interchange',
 'Cheltenham',
 'Chester Hill',
 'Circular Quay',
 'Major Interchange',
 'Clarendon',
 'Clyde',
 'Como',
 'Concord West',
 'Cronulla',
 'Croydon',
 'Denistone',
 'Domestic Airport',
 'Major Interchange',
 'Doonside',
 'Dulwich Hill',
 'East Hills',
 'East Richmond',
 'Eastwood',
 'Edgecliff',
 'Edmondson Park',
 'Emu Plains',
 'Engadine',
 'Epping',
 'Major Interchange',
 'Erskineville',
 'Fairfield',
 'Flemington',
 'Glenfield',
 'Major Interchange',
 'Gordon',
 'Granville',
 'Major Interchange',

###### Find the index from where the necessary data starts and ends and remove all the unnecessary data.

In [4]:
stations.index('Yennora')

191

In [5]:
del stations[193:]
stations

['Allawah',
 'Arncliffe',
 'Artarmon',
 'Ashfield',
 'Asquith',
 'Auburn',
 'Banksia',
 'Bankstown',
 'Bardwell Park',
 'Beecroft',
 'Belmore',
 'Berala',
 'Berowra',
 'Beverly Hills',
 'Bexley North',
 'Birrong',
 'Blacktown',
 'Major Interchange',
 'Bondi Junction',
 'Burwood',
 'Cabramatta',
 'Campbelltown',
 'Major Interchange',
 'Campsie',
 'Canley Vale',
 'Canterbury',
 'Caringbah',
 'Carlton',
 'Carramar',
 'Casula',
 'Central',
 'Major Interchange',
 'Chatswood',
 'Major Interchange',
 'Cheltenham',
 'Chester Hill',
 'Circular Quay',
 'Major Interchange',
 'Clarendon',
 'Clyde',
 'Como',
 'Concord West',
 'Cronulla',
 'Croydon',
 'Denistone',
 'Domestic Airport',
 'Major Interchange',
 'Doonside',
 'Dulwich Hill',
 'East Hills',
 'East Richmond',
 'Eastwood',
 'Edgecliff',
 'Edmondson Park',
 'Emu Plains',
 'Engadine',
 'Epping',
 'Major Interchange',
 'Erskineville',
 'Fairfield',
 'Flemington',
 'Glenfield',
 'Major Interchange',
 'Gordon',
 'Granville',
 'Major Interchange',

###### From this cell onwards, the extra parts of the data obtained such as the parts of the HTML code that are in the bold tags but not railway station names are removed to get pure data.

In [6]:
stations = set(stations)
stations

{'Allawah',
 'Arncliffe',
 'Artarmon',
 'Ashfield',
 'Asquith',
 'Auburn',
 'Banksia',
 'Bankstown',
 'Bardwell Park',
 'Beecroft',
 'Belmore',
 'Berala',
 'Berowra',
 'Beverly Hills',
 'Bexley North',
 'Birrong',
 'Blacktown',
 'Bondi Junction',
 'Burwood',
 'Cabramatta',
 'Campbelltown',
 'Campsie',
 'Canley Vale',
 'Canterbury',
 'Caringbah',
 'Carlton',
 'Carramar',
 'Casula',
 'Central',
 'Chatswood',
 'Cheltenham',
 'Chester Hill',
 'Circular Quay',
 'Clarendon',
 'Clyde',
 'Como',
 'Concord West',
 'Cronulla',
 'Croydon',
 'Denistone',
 'Domestic Airport',
 'Doonside',
 'Dulwich Hill',
 'East Hills',
 'East Richmond',
 'Eastwood',
 'Edgecliff',
 'Edmondson Park',
 'Emu Plains',
 'Engadine',
 'Epping',
 'Erskineville',
 'Fairfield',
 'Flemington',
 'Future Major Interchange',
 'Glenfield',
 'Gordon',
 'Granville',
 'Green Square',
 'Guildford',
 'Gymea',
 'Harris Park',
 'Heathcote',
 'Helensburgh',
 'Holsworthy',
 'Homebush',
 'Hornsby',
 'Hurlstone Park',
 'Hurstville',
 'Ingle

In [7]:
stations.discard('Major Interchange')
stations.discard('Future Major Interchange')
stations

{'Allawah',
 'Arncliffe',
 'Artarmon',
 'Ashfield',
 'Asquith',
 'Auburn',
 'Banksia',
 'Bankstown',
 'Bardwell Park',
 'Beecroft',
 'Belmore',
 'Berala',
 'Berowra',
 'Beverly Hills',
 'Bexley North',
 'Birrong',
 'Blacktown',
 'Bondi Junction',
 'Burwood',
 'Cabramatta',
 'Campbelltown',
 'Campsie',
 'Canley Vale',
 'Canterbury',
 'Caringbah',
 'Carlton',
 'Carramar',
 'Casula',
 'Central',
 'Chatswood',
 'Cheltenham',
 'Chester Hill',
 'Circular Quay',
 'Clarendon',
 'Clyde',
 'Como',
 'Concord West',
 'Cronulla',
 'Croydon',
 'Denistone',
 'Domestic Airport',
 'Doonside',
 'Dulwich Hill',
 'East Hills',
 'East Richmond',
 'Eastwood',
 'Edgecliff',
 'Edmondson Park',
 'Emu Plains',
 'Engadine',
 'Epping',
 'Erskineville',
 'Fairfield',
 'Flemington',
 'Glenfield',
 'Gordon',
 'Granville',
 'Green Square',
 'Guildford',
 'Gymea',
 'Harris Park',
 'Heathcote',
 'Helensburgh',
 'Holsworthy',
 'Homebush',
 'Hornsby',
 'Hurlstone Park',
 'Hurstville',
 'Ingleburn',
 'International Airpor

###### The words 'Railway Station' are added towards the end of the name of each station to get accurate locations.

In [8]:
stations = list(stations)
for i in range(len(stations)):
    stations[i] = stations[i] + " Railway Station"
stations

['Marayong Railway Station',
 'Penshurst Railway Station',
 'Rooty Hill Railway Station',
 'Tempe Railway Station',
 'Holsworthy Railway Station',
 'Wollstonecraft Railway Station',
 'Burwood Railway Station',
 'Kogarah Railway Station',
 'Mount Colah Railway Station',
 'Wynyard Railway Station',
 'Hurstville Railway Station',
 'Oatley Railway Station',
 'Dulwich Hill Railway Station',
 'Canley Vale Railway Station',
 'Ingleburn Railway Station',
 'Museum Railway Station',
 'Blacktown Railway Station',
 'Padstow Railway Station',
 'Banksia Railway Station',
 'Berowra Railway Station',
 'Heathcote Railway Station',
 'Vineyard Railway Station',
 'Yennora Railway Station',
 'Roseville Railway Station',
 'Wiley Park Railway Station',
 'Petersham Railway Station',
 'Waterfall Railway Station',
 'Edmondson Park Railway Station',
 'Casula Railway Station',
 'Bankstown Railway Station',
 'Stanmore Railway Station',
 'Waverton Railway Station',
 'Edgecliff Railway Station',
 'Kirrawee Railway S

In [9]:
for i in range(len(stations)):
    s = stations[i] 
    if (s[0] == ' '):
        stations[i] = s[1:]
stations

['Marayong Railway Station',
 'Penshurst Railway Station',
 'Rooty Hill Railway Station',
 'Tempe Railway Station',
 'Holsworthy Railway Station',
 'Wollstonecraft Railway Station',
 'Burwood Railway Station',
 'Kogarah Railway Station',
 'Mount Colah Railway Station',
 'Wynyard Railway Station',
 'Hurstville Railway Station',
 'Oatley Railway Station',
 'Dulwich Hill Railway Station',
 'Canley Vale Railway Station',
 'Ingleburn Railway Station',
 'Museum Railway Station',
 'Blacktown Railway Station',
 'Padstow Railway Station',
 'Banksia Railway Station',
 'Berowra Railway Station',
 'Heathcote Railway Station',
 'Vineyard Railway Station',
 'Yennora Railway Station',
 'Roseville Railway Station',
 'Wiley Park Railway Station',
 'Petersham Railway Station',
 'Waterfall Railway Station',
 'Edmondson Park Railway Station',
 'Casula Railway Station',
 'Bankstown Railway Station',
 'Stanmore Railway Station',
 'Waverton Railway Station',
 'Edgecliff Railway Station',
 'Kirrawee Railway S

###### Now that we have a list of the railway stations in Sydney without any garbage and perfectly cleaned, we can get the geographical locations of all the stations.

In [ ]:
geolocator = Nominatim(user_agent="stations_explorer")
lats=[]
longs=[]
address=[]
not_done=[]
for i in range(len(stations)):
    x = stations[i]+', Sydney, Australia'
    try:
        location = geolocator.geocode(x)
        if location:
            lats.append(location.latitude)
            longs.append(location.longitude)
        if location is None:
            lats.append(np.nan)
            longs.append(np.nan)
    except GeocoderTimedOut as e:
        not_done.append(i)
        lats.append(np.nan)
        longs.append(np.nan)

###### Convert all the names and locations into a dataframe.

In [ ]:
sydney_stations = pd.DataFrame(columns=['Station',"Latitude",'Longitude'])
sydney_stations['Station'] = stations
sydney_stations['Latitude'] = lats
sydney_stations['Longitude'] = longs
sydney_stations

###### Convert the dataframe to a csv file.

In [ ]:
sydney_stations.sort_values(by = ["Station"], inplace = True)
sydney_suburbs.dropna(inplace = True)

In [ ]:
sydney_stations

In [ ]:
sydney_stations.to_csv(r'sydney_stations.csv', index = False)